In [ ]:
import uproot
import awkward as ak

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

scale = 1.3

plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=scale*20)
plt.rc("xtick", labelsize=scale*16, top=True, direction="in")
plt.rc("ytick", labelsize=scale*16, right=True, direction="in")
plt.rc("axes", titlesize=scale*20)
plt.rc("legend", fontsize=14)

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
    print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
    sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    for f in font_files:
        font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
from rpad import rpad

os.makedirs("tmp", exist_ok=True)

dssds = ["U3", "U4", "U5"]
ids = [1, 2, 3]

In [ ]:
def f(v, a, b):
    return a*(1 + b*(1 - np.cos(v))/np.cos(v))

V = np.linspace(0, np.pi/4, 200)

In [ ]:
v, Edepmax, Eplow, Epupp, Edeplow = np.loadtxt("deadarea_U3.dat", unpack=True)
Edepmax /= 1e3
Eplow /= 1e3
Epupp /= 1e3
Edeplow /= 1e3

In [ ]:
resplow = curve_fit(f, v, Eplow, p0=(np.min(Eplow), 1.))
respupp = curve_fit(f, v, Epupp, p0=(np.min(Epupp), 1.))

In [ ]:
resdeplow = curve_fit(f, v, Edeplow, p0=(np.min(Edeplow), 1.))
resdepmax = curve_fit(f, v, Edepmax, p0=(np.min(Edepmax), 1.))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(2*6.4, 4.8))
plt.subplots_adjust(wspace=0.01)
plt.sca(axes[0])
plt.plot(np.rad2deg(v), Eplow, 'k.', ms=6, label='Punch through tabulation')
plt.plot(np.rad2deg(V), f(V, *resplow[0]), 'k-', lw=0.8, label='Punch through parametrisation')
plt.plot(np.rad2deg(v), Epupp, 'kx', ms=5, label='Reach through tabulation')
plt.plot(np.rad2deg(V), f(V, *respupp[0]), 'k', lw=0.8, ls=(0, (5, 5)), label='Reach through parametrisation')
plt.fill_between(np.rad2deg(V), f(V, *resplow[0]), f(V, *respupp[0]), color='whitesmoke')
plt.annotate("Dead zone", xy=(1, 2.5), xycoords='data', 
             xytext=(1, 2.7), textcoords='data', size=16,
             arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=110", lw=1.4, mutation_scale=15), 
             horizontalalignment='left', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='whitesmoke'))
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{p}}$ (MeV)')
plt.xticks(np.arange(0, 50, 10))
plt.xticks(np.arange(0, 50, 5), minor=True)
plt.yticks(np.arange(2.2, 3.4, 0.2))
plt.yticks(np.arange(2.2, 3.4, 0.05), minor=True)
plt.ylim(2.2, 3.2)
plt.legend()
plt.sca(axes[1])
plt.plot(np.rad2deg(v), Edepmax, 'k.', ms=6, label='Punch through tabulation')
plt.plot(np.rad2deg(V), f(V, *resdepmax[0]), 'k-', lw=0.8, label='Punch through parametrisation')
plt.plot(np.rad2deg(v), Edeplow, 'kx', ms=5, label='Reach through tabulation')
plt.plot(np.rad2deg(V), f(V, *resdeplow[0]), 'k', lw=0.8, ls=(0, (5, 5)), label='Reach through parametrisation')
plt.fill_between(np.rad2deg(V), f(V, *resdeplow[0]), f(V, *resdepmax[0]), color='whitesmoke')
plt.annotate("Spurious zone", xy=(1, 2.4), xycoords='data', 
             xytext=(1, 2.7), textcoords='data', size=16,
             arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=110", lw=1.4, mutation_scale=15), 
             horizontalalignment='left', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='whitesmoke'))
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{dep}}$ (MeV)')
axes[1].yaxis.tick_right()
axes[1].yaxis.set_ticks_position('both')
axes[1].yaxis.set_label_position("right")
plt.xticks(np.arange(0, 50, 10))
plt.xticks(np.arange(0, 50, 5), minor=True)
plt.yticks(np.arange(2.2, 3.4, 0.2))
plt.yticks(np.arange(2.2, 3.4, 0.05), minor=True)
plt.ylim(2.2, 3.2)
plt.legend()
plt.savefig('Ep_vs_angle_and_Edep_vs_angle.pdf', bbox_inches='tight')
plt.show()
print("Dead zone minimum    :", np.min(Eplow))
print("Dead zone maximum    :", np.max(Epupp))
print("Spurious zone minimum:", np.min(Edeplow))
print("Spurious zone maximum:", np.max(Edepmax))

In [ ]:
v, Edepmax, Eplow, Epupp, Edeplow = np.loadtxt("deadarea_U4.dat", unpack=True)
Edepmax /= 1e3
Eplow /= 1e3
Epupp /= 1e3
Edeplow /= 1e3

In [ ]:
resplow = curve_fit(f, v, Eplow, p0=(np.min(Eplow), 1.))
respupp = curve_fit(f, v, Epupp, p0=(np.min(Epupp), 1.))

In [ ]:
resdeplow = curve_fit(f, v, Edeplow, p0=(np.min(Edeplow), 1.))
resdepmax = curve_fit(f, v, Edepmax, p0=(np.min(Edepmax), 1.))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(2*6.4, 4.8), sharey=True)
plt.subplots_adjust(wspace=0.01)
plt.sca(axes[0])
plt.plot(np.rad2deg(v), Eplow, 'k.', ms=6, label='Punch through tabulation')
plt.plot(np.rad2deg(V), f(V, *resplow[0]), 'k-', lw=0.8, label='Punch through parametrisation')
plt.plot(np.rad2deg(v), Epupp, 'kx', ms=5, label='Reach through tabulation')
plt.plot(np.rad2deg(V), f(V, *respupp[0]), 'k', lw=0.8, ls=(0, (5, 5)), label='Reach through parametrisation')
plt.fill_between(np.rad2deg(V), f(V, *resplow[0]), f(V, *respupp[0]), color='whitesmoke')
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{p}}$ (MeV)')
#plt.xticks(np.arange(0, 50, 10))
#plt.xticks(np.arange(0, 50, 5), minor=True)
#plt.yticks(np.arange(2.2, 3.4, 0.2))
#plt.yticks(np.arange(2.2, 3.4, 0.05), minor=True)
#plt.ylim(2.2, 3.2)
plt.legend()
plt.sca(axes[1])
plt.plot(np.rad2deg(v), Edepmax, 'k.', ms=6, label='Punch through tabulation')
plt.plot(np.rad2deg(V), f(V, *resdepmax[0]), 'k-', lw=0.8, label='Punch through parametrisation')
plt.plot(np.rad2deg(v), Edeplow, 'kx', ms=5, label='Reach through tabulation')
plt.plot(np.rad2deg(V), f(V, *resdeplow[0]), 'k', lw=0.8, ls=(0, (5, 5)), label='Reach through parametrisation')
plt.fill_between(np.rad2deg(V), f(V, *resdeplow[0]), f(V, *resdepmax[0]), color='whitesmoke')
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{dep}}$ (MeV)')
axes[1].yaxis.tick_right()
axes[1].yaxis.set_ticks_position('both')
axes[1].yaxis.set_label_position("right")
plt.xticks(np.arange(0, 50, 10))
plt.xticks(np.arange(0, 50, 5), minor=True)
#plt.yticks(np.arange(2.2, 3.4, 0.2))
#plt.yticks(np.arange(2.2, 3.4, 0.05), minor=True)
#plt.ylim(2.2, 3.2)
plt.legend()
#plt.savefig('Ep_vs_angle_and_Edep_vs_angle.pdf', bbox_inches='tight')
plt.show()
print("Dead zone minimum    :", np.min(Eplow))
print("Dead zone maximum    :", np.max(Epupp))
print("Spurious zone minimum:", np.min(Edeplow))
print("Spurious zone maximum:", np.max(Edepmax))

In [ ]:
v, Edepmax, Eplow, Epupp, Edeplow = np.loadtxt("deadarea_U4_33um.dat", unpack=True)
Edepmax /= 1e3
Eplow /= 1e3
Epupp /= 1e3
Edeplow /= 1e3

In [ ]:
resplow = curve_fit(f, v, Eplow, p0=(np.min(Eplow), 1.))
respupp = curve_fit(f, v, Epupp, p0=(np.min(Epupp), 1.))

In [ ]:
resdeplow = curve_fit(f, v, Edeplow, p0=(np.min(Edeplow), 1.))
resdepmax = curve_fit(f, v, Edepmax, p0=(np.min(Edepmax), 1.))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(2*6.4, 4.8), sharey=True)
plt.subplots_adjust(wspace=0.01)
plt.sca(axes[0])
plt.plot(np.rad2deg(v), Eplow, 'k.', ms=6, label='Punch through tabulation')
plt.plot(np.rad2deg(V), f(V, *resplow[0]), 'k-', lw=0.8, label='Punch through parametrisation')
plt.plot(np.rad2deg(v), Epupp, 'kx', ms=5, label='Reach through tabulation')
plt.plot(np.rad2deg(V), f(V, *respupp[0]), 'k', lw=0.8, ls=(0, (5, 5)), label='Reach through parametrisation')
plt.fill_between(np.rad2deg(V), f(V, *resplow[0]), f(V, *respupp[0]), color='whitesmoke')
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{p}}$ (MeV)')
#plt.xticks(np.arange(0, 50, 10))
#plt.xticks(np.arange(0, 50, 5), minor=True)
#plt.yticks(np.arange(2.2, 3.4, 0.2))
#plt.yticks(np.arange(2.2, 3.4, 0.05), minor=True)
#plt.ylim(2.2, 3.2)
plt.legend()
plt.sca(axes[1])
plt.plot(np.rad2deg(v), Edepmax, 'k.', ms=6, label='Punch through tabulation')
plt.plot(np.rad2deg(V), f(V, *resdepmax[0]), 'k-', lw=0.8, label='Punch through parametrisation')
plt.plot(np.rad2deg(v), Edeplow, 'kx', ms=5, label='Reach through tabulation')
plt.plot(np.rad2deg(V), f(V, *resdeplow[0]), 'k', lw=0.8, ls=(0, (5, 5)), label='Reach through parametrisation')
plt.fill_between(np.rad2deg(V), f(V, *resdeplow[0]), f(V, *resdepmax[0]), color='whitesmoke')
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{dep}}$ (MeV)')
axes[1].yaxis.tick_right()
axes[1].yaxis.set_ticks_position('both')
axes[1].yaxis.set_label_position("right")
plt.xticks(np.arange(0, 50, 10))
plt.xticks(np.arange(0, 50, 5), minor=True)
#plt.yticks(np.arange(2.2, 3.4, 0.2))
#plt.yticks(np.arange(2.2, 3.4, 0.05), minor=True)
#plt.ylim(2.2, 3.2)
plt.legend()
#plt.savefig('Ep_vs_angle_and_Edep_vs_angle.pdf', bbox_inches='tight')
plt.show()
print("Dead zone minimum    :", np.min(Eplow))
print("Dead zone maximum    :", np.max(Epupp))
print("Spurious zone minimum:", np.min(Edeplow))
print("Spurious zone maximum:", np.max(Edepmax))